<a href="https://colab.research.google.com/github/EmKelsall/ADS2002---Catheter-Placement-Project/blob/main/InitialModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

print(tf.__version__)

2.12.0


## Preparing the Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
filenames = os.listdir("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train")

In [7]:
train_imgs = np.empty((0,2048, 2500))
FileIDs = []

#getting the first 100 images
for i in range(100):
  #Making sure the images are the same pixel size
  img = plt.imread(f"/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train/{filenames[i]}")
  img_shape = img.shape

  if img_shape == (2048, 2500):
    #Save filename for later
    FileIDs.append(filenames[i])

    #add extra dimension so we can concatenate
    img = np.expand_dims(img,(0))

    #adding to train_imgs array
    train_imgs = np.concatenate((train_imgs, img), axis = 0)

train_imgs.shape

(48, 2048, 2500)

In [9]:
#extracting just the CVC labels
train_data = pd.read_csv("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train.csv")
CVC_train = train_data[["StudyInstanceUID","CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
CVC_train.set_index('StudyInstanceUID', inplace=True)
CVC_train.head()

,CVC - Abnormal,CVC - Borderline,CVC - Normal
StudyInstanceUID,,,
1.2.826.0.1.3680043.8.498.26697628953273228189375557799582420561,0,0,0
1.2.826.0.1.3680043.8.498.46302891597398758759818628675365157729,0,0,1
1.2.826.0.1.3680043.8.498.23819260719748494858948050424870692577,0,1,0
1.2.826.0.1.3680043.8.498.68286643202323212801283518367144358744,1,0,0
1.2.826.0.1.3680043.8.498.10050203009225938259119000528814762175,0,0,1


In [10]:
#cleaning up the labels
UID = []
for i in range(len(FileIDs)):
  UID.append(FileIDs[i].replace(".jpg",""))

UID[0:10]

['1.2.826.0.1.3680043.8.498.96315500926641444675409331764690933307',
 '1.2.826.0.1.3680043.8.498.96450515721778915344219421534269786290',
 '1.2.826.0.1.3680043.8.498.96359554498564333142663951388648557789',
 '1.2.826.0.1.3680043.8.498.96447675290915183668080373345495953473',
 '1.2.826.0.1.3680043.8.498.96279880013483713441521686300695558821',
 '1.2.826.0.1.3680043.8.498.96349754766067875463134738379718157835',
 '1.2.826.0.1.3680043.8.498.96406955712119661656412685705445497931',
 '1.2.826.0.1.3680043.8.498.96346896468102014556173811602388995798',
 '1.2.826.0.1.3680043.8.498.96332366015730959061321671670587275158',
 '1.2.826.0.1.3680043.8.498.96365165965335417502443200533971674013']

In [15]:
train_labels = np.empty((0,3))
data = []
for ID in UID:
    data.append(CVC_train.loc[ID])

train_labels_df = pd.DataFrame(data)

train_labels_df = train_labels_df.reindex(UID)

train_labels = train_labels_df.values

## Making and training the model

In [21]:
if np.all(train_labels[1] == [0,0,1]):
  print('hello')

hello


In [35]:
def get_allignment(labels):
  result = ""
  if np.any(labels == [2,2,1]):
    result += "Normal"

  elif np.any(labels == [2,1,2]):
    result += "Borderline"

  elif np.any(labels == [1,2,2]):
    result += "Abnormal"

  return result

In [36]:
#displaying 6 of the images in the train_imgs array
plt.figure(figsize=(12,9))
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_imgs[i], cmap=plt.cm.binary)
  plt.xlabel(get_allignment(train_labels[i]))
plt.show()

In [ ]:
#initiating the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2048, 2500)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8)
    ])

In [ ]:
#compling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#fitting the model (currently trying to fit the model uses all the available ram and causes the notebook to crash)
model.fit(train_imgs, train_labels, epochs=10)

Epoch 1/10
